In [1]:
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig
from PIL import Image
import requests
from pathlib import Path
from qwen_model import Draw

In [2]:
path = Path.home() / 'Downloads/Data/images'
files = [*path.glob('*.jpg')]
files[:2]

[PosixPath('/Users/kevinkhang/Downloads/Data/images/vn-11134207-7r98o-lzxk1jvunnvl91@resize_w900_nl.jpg'),
 PosixPath('/Users/kevinkhang/Downloads/Data/images/vn-11134207-7r98o-lvioho72yfm50d@resize_w900_nl.jpg')]

In [3]:
# load the processor
model_id = 'allenai/MolmoE-1B-0924' 
processor = AutoProcessor.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype='auto',
    device_map='auto'
)

# load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype='auto',
    device_map='auto'
)

# process the image and text
inputs = processor.process(
    images=[Image.open(files[0])],
    text="Describe this image."
)
inputs = {k: v.to(model.device).unsqueeze(0) for k, v in inputs.items()}

# generate output; maximum 200 new tokens; stop generation when <|endoftext|> is generated
output = model.generate_from_batch(
    inputs,
    GenerationConfig(max_new_tokens=200, stop_strings="<|endoftext|>"),
    tokenizer=processor.tokenizer
)
generated_tokens = output[0,inputs['input_ids'].size(1):]
generated_text = processor.tokenizer.decode(generated_tokens, skip_special_tokens=True)

print(generated_text)

# >>> This photograph captures a small black puppy, likely a Labrador or a similar breed,
#     sitting attentively on a weathered wooden deck. The deck, composed of three...


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPTNeoXTokenizer'. 
The class this function is called from is 'Qwen2TokenizerFast'.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


ValueError: The current `device_map` had weights offloaded to the disk. Please provide an `offload_folder` for them. Alternatively, make sure you have `safetensors` installed if the model you are using offers the weights in this format.